In [7]:
# 2D Crack propagation in a bended beam

 - @todo: Plot the residual value
 - @todo: State dependencies - recalculate upon material parameter change

In [8]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
ce = CrackPropagation(n_seg=250)
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
ce.sz_bd.cmm.trait_set(f_c = 33.3, f_t = 0.3 * 33.3 **(2/3), d_a = 16)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=600, B=250, L=3870)
ce.sz_cp.trait_set(x_00=ce.sz_bd.L * 0.3, n_m = 15)
ce.sz_cp.sz_ctr.trait_set(ell = 2)
ce.sz_bd.cross_section_layout.reinforcement.z_j[0] = 30
ce.sz_bd.cross_section_layout.reinforcement.A_j[0] = 615.44
ce.sz_bd.cross_section_layout.reinforcement.E_j[0] = 210000 / 100

\begin{align}
\sigma_\mathrm{F} = E w / L_\mathrm{eff}
\end{align}

In [9]:
bu.InteractiveWindow([ce, ce.sz_bd, ce.sz_bd.cross_section_layout, 
                      ce.sz_stress_profile, 
                      ce.sz_bd.cmm, ce.sz_bd.smm, ce.sz_cp, ce.sz_cp.sz_ctr]).interact()

<lambdifygenerated-40>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


<lambdifygenerated-40>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))
<lambdifygenerated-40>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [ ]:
import numpy as np
S_Lb = ce.sz_stress_profile.S_Lb
S_La = ce.sz_stress_profile.S_La
idx = np.argmax(S_Lb[...,0])

In [ ]:
S_Lb[idx,:], S_La[idx,:]

In [ ]:
ce.sz_cp.T_Mab[idx,...]

In [ ]:
np.einsum('i,i', S_Lb[idx,:], S_Lb[idx,:])

In [ ]:
np.einsum('i,i', S_La[idx,:], S_La[idx,:])

Todo:

 - include the reinforcement material law - partially done 
   (but for strain-based material law) - replace with the crack bridge law
 - transformation of the crack bridging force to the global coordinates
 - include the dowel effect 

In [ ]:
#ce.sz_cp._get_norm_n_vec_L()

In [ ]:
#ce.sz_cp._get_x_Ka()

In [ ]:
#ce.sz_stress_profile._get_S_La()

In [ ]:
#ce.sz_stress_profile._get_F_La()

In [ ]:
#ce.sz_stress_profile._get_F_ag()

In [ ]:
#ce.sz_stress_profile._get_u_Lb()

In [ ]:
#ce.sz_stress_profile._get_S_Lb()[:,1]

In [ ]:
ce.sz_stress_profile.u_La[...,1]

In [ ]:
ce.sz_cp.x_Ka

In [ ]:
ce.sz_stress_profile.u_La[...,1]

In [ ]:
ce.sz_stress_profile.normalized_def #[...,1]

In [ ]:
ce.sz_stress_profile.tau_exp

In [ ]:
ce.sz_stress_profile.F_Na[...,0]

In [ ]:
ce.sz_ctr._get_x_rot_ak()